In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(model = "openai/gpt-oss-20b")
llm_response  = llm.invoke("What is the capital of France?")
print(llm_response)

e:\Karamveer\learnings\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


content='The capital of France is **Paris**.' additional_kwargs={'reasoning_content': 'The user asks: "What is the capital of France?" They want a straightforward answer: Paris. Provide that.'} response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 78, 'total_tokens': 120, 'completion_time': 0.043277045, 'completion_tokens_details': {'reasoning_tokens': 24}, 'prompt_time': 0.005116795, 'prompt_tokens_details': None, 'queue_time': 0.047537254, 'total_time': 0.04839384}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_e99e93f2ac', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None, 'model_provider': 'groq'} id='lc_run--3d41dd04-bf5b-4e19-84dd-f4a95f7db965-0' usage_metadata={'input_tokens': 78, 'output_tokens': 42, 'total_tokens': 120, 'output_token_details': {'reasoning': 24}}


In [3]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
chain = llm | output_parser
parsed_response = chain.invoke("What is the capital of Germany?")
print(parsed_response)

The capital of Germany is Berlin.


In [4]:
from typing import List
from pydantic import BaseModel, Field

class RelocationDetails(BaseModel):
    city: str = Field(description="The city to which the person is relocating")
    country: str = Field(description="The country to which the person is relocating")
    from_city: str = Field(description="The city from which the person is relocating")
    from_country: str = Field(description="The country from which the person is relocating")
    recommended_neighborhoods: List[str] = Field(description="A list of recommended neighborhoods in the new city")
    average_cost_of_living: float = Field(description="The average monthly cost of living in the new city in USD")
    summary: str = Field(description="A brief summary of the relocation details")

relocation_text = """
A Candidate is living in london and is relocating to Berlin, Germany for a new job opportunity.
"""

structured_llm = llm.with_structured_output(RelocationDetails)
relocation_details = structured_llm.invoke(relocation_text)
print(relocation_details)

city='Berlin' country='Germany' from_city='London' from_country='United Kingdom' recommended_neighborhoods=['Mitte', 'Prenzlauer Berg', 'Friedrichshain'] average_cost_of_living=2500.0 summary='The candidate is moving from London, UK, to Berlin, Germany for a new job opportunity. Berlin offers a vibrant cultural scene, a lower cost of living compared to London, and a growing tech ecosystem. Recommended neighborhoods include Mitte for its central location and historic charm, Prenzlauer Berg for its family-friendly atmosphere and cafés, and Friedrichshain for its lively nightlife and creative vibe.'


In [5]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
  "Provide a brief summary about {topic}."
)
chain = prompt | llm | output_parser
result = chain.invoke({"topic": "the Eiffel Tower"})
print(result)

**Eiffel Tower – Quick Overview**

- **Location:** Champ de Mars, Paris, France  
- **Designer:** Gustave Eiffel (engineering firm) with architect Maurice Koechlin & engineer Émile Nouguier  
- **Construction:** 1887‑1889 for the 1889 Exposition Universelle (World’s Fair) celebrating the 100th anniversary of the French Revolution  
- **Height:** 330 m (1,083 ft) at its peak (including antennas), originally 300 m (984 ft)  
- **Structure:** Wrought‑iron lattice, 18,038 individual metal parts, 2,665 rivets, 7,300 tons of iron  
- **Purpose:** Initially a temporary exhibit; later a symbol of French industrial prowess and a global icon  
- **Tourism:** Three public platforms (1st, 2nd, 3rd levels), restaurants, observation decks; over 7 million visitors annually (pre‑COVID)  
- **Cultural Impact:** Frequently used in film, art, and as a symbol of Paris; remains the most-visited paid monument in the world.


In [6]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(content="You are a helpful assistant that provides concise information."),
    HumanMessage(content="Tell me a fun fact about the Great Wall of China.")
]

result = llm.invoke(messages)
print(result)

template = ChatPromptTemplate([
  ("system", "You are a knowledgeable assistant."),
  ("human", "Explain the significance of {event} in history.")
])

chain = template | llm
response = chain.invoke({"event": "the fall of the Berlin Wall"})
print(response)

content='A fun fact: the Great Wall’s total length—including all its branches and sections—is about **21,196\u202fkm (13,170\u202fmi)**, roughly **5½ times the Earth’s circumference**.' additional_kwargs={'reasoning_content': 'We need to give a concise fun fact about the Great Wall. The instructions: "You are a helpful assistant that provides concise information." So keep it brief. Provide one fun fact. Let\'s think of a fun fact: The Great Wall isn\'t a single wall but many walls, the longest section is about 9,000 kilometers, it\'s not visible from space, the wall is built with bricks and tamped earth, etc. Fun fact: It was built over centuries, the wall has more than 5 million bricks, or the wall can be seen from the moon? No, that\'s false. Another: The Great Wall was built to protect against Mongols, but the first walls were built in 7th century BC. Or the wall\'s total length, including all branches, is about 21,196 km (13,170 miles). Fun fact: The Great Wall is not a single cont

In [7]:
from langchain_community.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from typing import List
from langchain_core.documents import Document
import os

def load_documents(folder_path: str) -> List[Document]:
    documents = []
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if filename.endswith('.pdf'):
            loader = PyPDFLoader(file_path)
        elif filename.endswith('.docx'):
            loader = Docx2txtLoader(file_path)
        else:
            print(f"Unsupported file type: {filename}")
            continue
        documents.extend(loader.load())
    return documents

folder_path = "./documents"
documents = load_documents(folder_path)
print(f"Loaded {len(documents)} documents from the folder.")


Loaded 94 documents from the folder.


In [8]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, length_function=len)
splits = text_splitter.split_documents(documents)
print(f"Split into {len(splits)} chunks.")

Split into 137 chunks.


In [9]:
print(documents[0])

page_content='HackRF
Great Scott Gadgets
Jul 17, 2024' metadata={'producer': 'pdfTeX-1.40.22', 'creator': 'LaTeX with hyperref', 'creationdate': '2024-07-17T17:48:14+00:00', 'author': 'Great Scott Gadgets', 'title': 'HackRF', 'subject': '', 'keywords': '', 'moddate': '2024-07-17T17:48:14+00:00', 'trapped': '/False', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.22 (TeX Live 2022/dev/Debian) kpathsea version 6.3.4/dev', 'source': './documents\\DOC052841344.pdf', 'total_pages': 94, 'page': 0, 'page_label': '1'}


In [10]:
print(splits[1])
print(splits[0].metadata)

page_content='USER DOCUMENTATION
1 Getting Help 1
2 FAQ 3
2.1 What is the Transmit Power of HackRF? . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 3
2.2 What is the Receive Power of HackRF? . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 3
2.3 What is the minimum signal power level that can be detected by HackRF? . . . . . . . . . . . . . . . 4
2.4 Is HackRF full-duplex? . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 4
2.5 Why isn’t HackRF One full-duplex? . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 4
2.6 How could the HackRF One design be changed to make it full-duplex? . . . . . . . . . . . . . . . . 4
2.7 What is the big spike in the center of my received spectrum? . . . . . . . . . . . . . . . . . . . . . . 5
2.8 How do I deal with the big spike in the middle of my spectrum? . . . . . . . . . . . . . . . . . . . . 5' metadata={'producer': 'pdfTeX-1.40.22', 'creator': 'LaTeX with hyperr

In [11]:
from langchain_community.embeddings.sentence_transformer import SentenceTransformerEmbeddings

embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
document_embeddings= embedding_function.embed_documents([split.page_content for split in splits])
print(document_embeddings[0][:5])

C:\Users\DELL-7373\AppData\Local\Temp\ipykernel_3740\2713969797.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")


[-0.16012603044509888, 0.00766132352873683, -0.04513724893331528, -0.07161771506071091, 0.005821555387228727]


In [12]:
from langchain_chroma import Chroma

collection_name = "Hackrf"

vector_store = Chroma.from_documents(
  collection_name = collection_name,
  documents = splits,
  embedding=embedding_function,
  persist_directory="./chroma_db" 
)

print("Vector store created and documents added.")

Vector store created and documents added.


In [13]:
query = "What are the key features of HackRF?"
search_results = vector_store.similarity_search(query, k=3)
print(f"\nTop 2 most relevant chunks for the query: '{query}'\n")
for i, result in enumerate(search_results, 1):
    print(f"Result {i}:")
    print(f"Source: {result.metadata.get('source', 'Unknown')}")
    print(f"Content: {result.page_content}")
    print()


Top 2 most relevant chunks for the query: 'What are the key features of HackRF?'

Result 1:
Source: ./documents\DOC052841344.pdf
Content: CHAPTER
TWENTYTWO
INSTALLING HACKRF SOFTWARE
HackRF software includes HackRF Tools and libhackrf. HackRF Tools are the commandline utilities that let you
interact with your HackRF. libhackrf is a low level library that enables software on your computer to operate with
HackRF.
22.1 Install Using Package Managers
Unless developing or testing new features for HackRF, we highly recommend that most users use build systems or
package managers provided for their operating system.Our suggested operating system for use with HackRF is
Ubuntu.
22.1.1 FreeBSD
You can use the binary package:# pkg install hackrf
You can also build and install from ports:
# cd /usr/ports/comms/hackrf
# make install
22.1.2 Linux: Arch
pacman -S hackrf
22.1.3 Linux: Fedora / Red Hat
sudo dnf install hackrf -y
63

Result 2:
Source: ./documents\DOC052841344.pdf
Content: CHAPTER
TWENTY

In [14]:
retriever = vector_store.as_retriever(search_kwargs={"k": 2})
retriever_results = retriever.invoke("Features of Hackrf")
print(retriever_results)

[Document(id='83a0e35e-f127-4e4e-baf2-d93211e08a5c', metadata={'creationdate': '2024-07-17T17:48:14+00:00', 'total_pages': 94, 'page_label': '63', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.22 (TeX Live 2022/dev/Debian) kpathsea version 6.3.4/dev', 'keywords': '', 'producer': 'pdfTeX-1.40.22', 'moddate': '2024-07-17T17:48:14+00:00', 'creator': 'LaTeX with hyperref', 'page': 68, 'trapped': '/False', 'author': 'Great Scott Gadgets', 'source': './documents\\DOC052841344.pdf', 'subject': '', 'title': 'HackRF'}, page_content='CHAPTER\nTWENTYTWO\nINSTALLING HACKRF SOFTWARE\nHackRF software includes HackRF Tools and libhackrf. HackRF Tools are the commandline utilities that let you\ninteract with your HackRF. libhackrf is a low level library that enables software on your computer to operate with\nHackRF.\n22.1 Install Using Package Managers\nUnless developing or testing new features for HackRF, we highly recommend that most users use build systems or\npackage managers pr

In [15]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

template = """Answer the question based only on the following context:
{context}
Question: {question}
Answer: """

prompt = ChatPromptTemplate.from_template(template)

def docs2str(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | docs2str, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)


In [16]:
question = "Can you list some applications of HackRF in wireless communication?"
response = rag_chain.invoke(question)
print(f"Question: {question}")
print(f"Answer: {response}")

Question: Can you list some applications of HackRF in wireless communication?
Answer: Based on the information provided, HackRF One can be used for a variety of wireless‑communication tasks, including:

1. **Transmission and reception of radio signals** across the 1 MHz to 6 GHz band.  
2. **Testing and development of modern and next‑generation radio technologies.**  
3. **Operating as a USB peripheral** for quick integration with a computer.  
4. **Programming for stand‑alone operation** to run independent radio applications.

These capabilities make it a versatile tool for experimenting with and building wireless communication systems.


In [ ]:
from langchain_core.prompts import MessagesPlaceholder
from langchain_classic.chains import create_history_aware_retriever
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

contextualize_q_system_prompt = """
Given a chat history and the latest user question
which might reference context in the chat history,
formulate a standalone question which can be understood
without the chat history. Do NOT answer the question,
just reformulate it if needed and otherwise return it as is.
"""

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)

contextualize_chain = contextualize_q_prompt | llm | StrOutputParser()
print(contextualize_chain.invoke({"input": "What are its applications?", "chat_history": []}))


What are the applications of the topic you are referring to?


In [ ]:
from langchain_classic.chains import create_retrieval_chain

history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

qa_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant. Use the following context to answer the user's question."),
    ("system", "Context: {context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("human", "{input}")
])

question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)
rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


In [22]:
from langchain_core.messages import HumanMessage, AIMessage

chat_history = []
question1 = "Whats hackrf?"
answer1 = rag_chain.invoke({"input": question1, "chat_history": chat_history})['answer']
chat_history.extend([
    HumanMessage(content=question1),
    AIMessage(content=answer1)
])

print(f"Human: {question1}")
print(f"AI: {answer1}\n")

question2 = "Where are its applications?"
answer2 = rag_chain.invoke({"input": question2, "chat_history": chat_history})['answer']
chat_history.extend([
    HumanMessage(content=question2),
    AIMessage(content=answer2)
])

print(f"Human: {question2}")
print(f"AI: {answer2}")


Human: Whats hackrf?
AI: **HackRF** is an open‑source software‑defined radio (SDR) platform that lets you transmit and receive radio signals over a wide frequency range (from 1 MHz up to 6 GHz). It consists of two main parts:

| Part | What it is | What it does |
|------|------------|--------------|
| **Hardware** | A small, inexpensive daughterboard that plugs into a host computer via USB. | Provides the RF front‑end: antenna input, amplification, frequency mixing, and digital conversion (ADC/DAC). |
| **Software** | *HackRF Tools* (command‑line utilities) and *libhackrf* (a C library). | Allows you to control the device, capture or transmit samples, run signal‑processing scripts, and integrate the hardware into other SDR projects. |

### Key features

* **Frequency range:** 1 MHz – 6 GHz (theoretically up to 6 GHz; practical limits around 2 GHz‑5 GHz depending on the antenna and environment).
* **Bandwidth:** Up to 20 MHz continuous, 10 MHz effective after processing.
* **Sampling ra